<a href="https://colab.research.google.com/github/zeiadh20/EA/blob/main/Copy_of_notebookd31905d13f.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import tensorflow as tf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Define Gomoku game mechanics
BOARD_SIZE = 15
WINNING_LENGTH = 5
EMPTY = 0
PLAYER_X = 1
PLAYER_O = 2

In [ ]:
# Define neural network architecture
class GomokuNeuralNetwork(tf.keras.Model):
    def __init__(self):
        super(GomokuNeuralNetwork, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(BOARD_SIZE, BOARD_SIZE, 1))
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.dense2 = tf.keras.layers.Dense(BOARD_SIZE**2, activation='softmax')

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

In [ ]:
# Genetic Algorithm parameters
population_size = 10
mutation_rate = 0.1

# Initialize population of players
players = [GomokuNeuralNetwork() for _ in range(population_size)]

In [ ]:
# Define evaluation function
def evaluate(player):
    # Simulate games between the player and a random opponent and return win rate
    wins = 0
    total_games = 3
    for _ in range(total_games):
        random_player = random.choice(players)  # Select a random opponent
        winner = play_game(player, random_player)
        if winner == PLAYER_X:
            wins += 1
    return wins / total_games


In [ ]:
def evaluate1(player):
    # Simulate games between the player and all other players and return win rate against each player
    wins_count = 0
    total_games = 0
    for opponent in players:
        if opponent != player:  # Ensure players are different
            for _ in range(3):  # Play 3 games against each opponent
                winner = play_game(player, opponent)
                total_games += 1
                if winner == PLAYER_X:
                    wins_count += 1
    return wins_count / total_games if total_games > 0 else 0

# Example usage:
for player in players:
    win_rate = evaluate1(player)
    print(f"Win rate for {player}: {win_rate}")


In [ ]:
def play_game(player1, player2):
    # Simulate a game of Gomoku between two players
    board = np.zeros((BOARD_SIZE, BOARD_SIZE), dtype=int)
    current_player = PLAYER_X
    while True:
        # Get move from current player
        if current_player == PLAYER_X:
            move = select_move(player1, board)
        else:
            move = select_move(player2, board)
        # Make move
        board[move] = current_player
        # Check for winner
        if check_winner(board, move, current_player):
            return current_player  # Return the winning player
        # Check for draw
        if np.all(board != 0):
            return None  # Return None for draw
        # Switch players
        current_player = PLAYER_O if current_player == PLAYER_X else PLAYER_X


In [ ]:
# # Define game simulation function
# def play_game(player1, player2):
#     # Simulate a game of Gomoku between two players
#     board = np.zeros((BOARD_SIZE, BOARD_SIZE), dtype=int)
#     current_player = PLAYER_X
#     while True:
#         # Get move from current player
#         if current_player == PLAYER_X:
#             move = select_move(player1, board)
#         else:
#             move = select_move(player2, board)
#         # Make move
#         board[move] = current_player
#         # Check for winner
#         if check_winner(board, move, current_player):
#             return current_player
#         # Switch players
#         current_player = PLAYER_O if current_player == PLAYER_X else PLAYER_X

In [ ]:
def select_move(player, board):
    # Use neural network to select move
    input_board = np.expand_dims(np.expand_dims(board, axis=0), axis=-1)
    probabilities = player(input_board).numpy().reshape((BOARD_SIZE, BOARD_SIZE))
    valid_moves = np.where(board == EMPTY)
    valid_probabilities = probabilities[valid_moves]

    # Check if all probabilities are zero
    if np.sum(valid_probabilities) == 0:
        # Select a random move
        move_index = np.random.choice(len(valid_moves[0]))
    else:
        # Normalize probabilities
        valid_probabilities /= np.sum(valid_probabilities)
        # Select move based on probabilities
        move_index = np.random.choice(len(valid_probabilities), p=valid_probabilities)

    move = (valid_moves[0][move_index], valid_moves[1][move_index])
    return move


In [ ]:
# Define function to check for winner
def check_winner(board, move, player):
    row, col = move
    directions = [(1, 0), (0, 1), (1, 1), (1, -1)]
    for dr, dc in directions:
        count = 1
        for i in range(1, WINNING_LENGTH):
            r = row + i * dr
            c = col + i * dc
            if 0 <= r < BOARD_SIZE and 0 <= c < BOARD_SIZE and board[r, c] == player:
                count += 1
            else:
                break
        for i in range(1, WINNING_LENGTH):
            r = row - i * dr
            c = col - i * dc
            if 0 <= r < BOARD_SIZE and 0 <= c < BOARD_SIZE and board[r, c] == player:
                count += 1
            else:
                break
        if count >= WINNING_LENGTH:
            return True
    return False

In [ ]:
def crossover(parent1, parent2):
    # Create child models with the same architecture as the parents
    child1 = GomokuNeuralNetwork()
    child2 = GomokuNeuralNetwork()

    # Build child models to initialize their weights
    _ = child1(np.zeros((1, BOARD_SIZE, BOARD_SIZE, 1)))  # Use a dummy input to build the model
    _ = child2(np.zeros((1, BOARD_SIZE, BOARD_SIZE, 1)))  # Use a dummy input to build the model

    # Copy weights from parents to children
    for i, (layer1, layer2) in enumerate(zip(parent1.layers, parent2.layers)):
        if isinstance(layer1, tf.keras.layers.Conv2D):
            child1.layers[i].set_weights(layer1.get_weights())
            child2.layers[i].set_weights(layer2.get_weights())
        elif isinstance(layer1, tf.keras.layers.Dense):
            child1.layers[i].set_weights(layer1.get_weights())
            child2.layers[i].set_weights(layer2.get_weights())

    return child1, child2


In [ ]:
# Define mutation function
def mutate(child, mutation_rate):
    for layer in child.layers:
        if isinstance(layer, tf.keras.layers.Dense):
            mutation_mask = np.random.rand(*layer.get_weights()[0].shape) < mutation_rate
            layer.set_weights([layer.get_weights()[0] + mutation_mask * np.random.randn(*layer.get_weights()[0].shape),
                                layer.get_weights()[1]])
    return child

In [ ]:
def tournament_selection(players, fitness_scores, tournament_size):
    tournament_indices = np.random.choice(range(len(players)), size=tournament_size, replace=False)
    tournament_players = [players[i] for i in tournament_indices]
    tournament_fitness_scores = [fitness_scores[i] for i in tournament_indices]
    return tournament_players[np.argmax(tournament_fitness_scores)]

In [ ]:
generations = 30
count=0
for generation in range(generations):
    # Evaluate fitness of each player
    fitness_scores = np.array([evaluate1(player) for player in players])
    count+=1
    print(f"Generation {count} Done")
    # Select parents for crossover
    parent_indices = np.random.choice(range(population_size), size=population_size, replace=True)
    # Create offspring by crossover and mutation
    for i in range(0, population_size, 2):
        parent1 = tournament_selection(players, fitness_scores, tournament_size=3)
        parent2 = tournament_selection(players, fitness_scores, tournament_size=3)

        child1, child2 = crossover(parent1, parent2)

        # Perform mutation
        child1 = mutate(child1, mutation_rate)
        child2 = mutate(child2, mutation_rate)

        # Replace parents with offspring
        players[i] = child1
        players[i+1] = child2

In [ ]:

# # Evaluate best player against a random opponent
# best_player = max(players, key=evaluate1)
# random_player = GomokuNeuralNetwork()
# print("Win rate:", evaluate1(best_player))


In [ ]:
print(best_player)

In [ ]:
players.append(best_player)

In [ ]:
print(players)

In [ ]:
best_player2 = max(players, key=evaluate1)
random_player = GomokuNeuralNetwork()
print("Win rate:", evaluate1(best_player2))


In [ ]:
print(best_player2)

In [ ]:
list=[]
for a in players:
    x= evaluate(a)
    list.append(x)
print(list)



In [ ]:
# Get the weights of the best player
best_player.get_weights()
